In [ ]:
# Instalar a biblioteca do cliente do Google e a biblioteca para autenticação
!pip install --upgrade google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from google.colab import auth
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Nome do arquivo JSON que você fez upload
# Certifique-se de que o nome do arquivo aqui corresponde ao nome do arquivo que você baixou e fez upload
SERVICE_ACCOUNT_FILE = 'mba-usp-469620-7ff99acd80ca.json' # <-- ATUALIZE AQUI COM O NOME DO SEU ARQUIVO JSON

# Adicione o arquivo JSON à pasta raiz do seu Colab ou atualize o caminho abaixo se o upload foi para outra pasta
# O caminho padrão após o upload é geralmente na raiz do ambiente Colab (/content/)

# Carregar as credenciais da conta de serviço
try:
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets.readonly'])
    print("Credenciais carregadas com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo de credenciais '{SERVICE_ACCOUNT_FILE}' não foi encontrado.")
    print("Por favor, verifique se você fez upload do arquivo JSON para o Colab")
    print("e se o nome do arquivo na variável SERVICE_ACCOUNT_FILE está correto.")
except Exception as e:
    print(f"Ocorreu um erro ao carregar as credenciais: {e}")

# Constrói o serviço Sheets
if 'credentials' in locals():
    service = build('sheets', 'v4', credentials=credentials)
    print("Serviço Google Sheets construído com sucesso!")
else:
    print("Não foi possível construir o serviço Google Sheets devido a erros de credenciais.")

Credenciais carregadas com sucesso!
Serviço Google Sheets construído com sucesso!


In [ ]:
# Passo 3: Ler dados da planilha usando a API do Google Sheets

# Atualize com o ID da sua planilha e o nome da aba (ou range)
# O SPREADSHEET_ID está na URL da sua planilha, entre /d/ e /edit
SPREADSHEET_ID = '1sTiUrfRTddQLssaYDR38ouGwfmkSUP0rVY4ssO2dc2A' # <-- ATUALIZE AQUI COM O ID DA SUA PLANILHA
RANGE_NAME = 'Respostas ao formulário 1!A1:AD42' # <-- ATUALIZE AQUI COM O NOME DA ABA E O RANGE DOS DADOS (ex: Sheet1!A1:D10),

try:
    # Chamada à API para obter os dados
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('Nenhum dado encontrado no range especificado.')
    else:
        print(f'Dados lidos com sucesso! Número de linhas: {len(values)}')
        # Os dados lidos estão na variável 'values' (lista de listas)

except Exception as e:
    print(f"Ocorreu um erro ao ler os dados da planilha: {e}")

Dados lidos com sucesso! Número de linhas: 42


In [ ]:
# Passo 4: Carregar dados em um DataFrame

import pandas as pd
import numpy as np
# A primeira linha da lista 'values' são os cabeçalhos
headers = values[0]
# As linhas restantes são os dados
data = values[1:]

if 'values' in locals() and values:
    # The first row of 'values' are the headers
    headers = values[0]
    # The remaining rows are the data
    data = values[1:]

    # Criar o DataFrame
    df_raw = pd.DataFrame(data, columns=headers)
    print("✅ DataFrame criado com sucesso!")

    # --- Limpar nomes das colunas em df_raw (remover espaços em branco no início/fim) ---
    # This step is still important to handle potential spaces in other columns not explicitly mapped
    df_raw.columns = df_raw.columns.str.strip()
    print("✅ Espaços removidos dos nomes das colunas em df_raw.")

pd.set_option('display.max_columns', None)

display(df_raw.head())

✅ DataFrame criado com sucesso!
✅ Espaços removidos dos nomes das colunas em df_raw.


,Carimbo de data/hora,"Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]","Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]","Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]","Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]",Em qual estado da federação você reside?,Há quanto tempo atua na área de tecnologia?,"Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Infraestrutura]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [QA]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [SI]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]","Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)",O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.,Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.,Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.,Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.,"Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)","Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória."
0,29/05/2025 14:25:27,,,,Pós-graduação,,Mulher(cis),,,,,Sao,7 a 10 anos,,,,,,,,,,Estag/Trainee,Interseccional (direcionada a mulheres negras)...,Neutro,Neutro,Discordo,Discordo,Salário competitivo,None
1,30/05/2025 

In [ ]:
# Mostrar o número de respostas e de perguntas
print(f"Total de respostas (participantes): {df_raw.shape[0]}")
print(f"Total de perguntas (colunas): {df_raw.shape[1]}")
print("\n")

Total de respostas (participantes): 41
Total de perguntas (colunas): 30




In [ ]:
# Analisar a frequência das respostas para o perfil demográfico e profissional
# Usando os nomes originais das colunas, como estão no seu arquivo bruto.
print("### Análise de Frequência - Perfil da Amostra ###")
print("-" * 40)

### Análise de Frequência - Perfil da Amostra ###
----------------------------------------


In [ ]:
# 1. Perfil de Gênero e Raça (com base nas colunas da matriz)
print("1. Perfil de Gênero e Raça:\n")
# Filtrando as colunas de forma mais precisa
genero_raca_cols = [
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]'
]

# Função de consolidação para extrair ambos gênero e raça
def consolidate_genero_raca_pair(row):
    for col in genero_raca_cols:
        value = row[col]
        # Verifica se a célula não está vazia e não é um espaço em branco
        if pd.notna(value) and str(value).strip() != '':
            raca = col.split('[')[-1].replace(']','').strip()
            genero = str(value).strip()
            return pd.Series([genero, raca])
    return pd.Series([np.nan, np.nan])

# Aplicar a função para criar as novas colunas
df_raw[['genero', 'raca']] = df_raw.apply(consolidate_genero_raca_pair, axis=1)

# Agora, fazemos a análise de frequência nas novas colunas
print("### Análise de Frequência - Perfil de Gênero e Raça ###")
print("-" * 40)
genero_counts = df_raw['genero'].value_counts()
raca_counts = df_raw['raca'].value_counts()
print("Distribuição de Gênero:")
print(genero_counts.to_string())
print("\n")
print("Distribuição de Raça:")
print(raca_counts.to_string())
print("\n")



1. Perfil de Gênero e Raça:

### Análise de Frequência - Perfil de Gênero e Raça ###
----------------------------------------
Distribuição de Gênero:
genero
Mulher(cis)      40
Não responder     1


Distribuição de Raça:
raca
Preta      21
Parda      12
Branca      6
Amarela     2




In [ ]:
import pandas as pd
import numpy as np

# --- Passo 1: Ler os dados brutos da sua planilha do Google Sheets ---
# df_raw já carregado da etapa anterior
# print(df_raw.head())

df_raw.columns = df_raw.columns.str.strip()

# --- Mapeamento completo das colunas corrigido ---
column_mapping = {
    'Carimbo de data/hora': 'timestamp',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]': 'idade_ate_25',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]': 'idade_26_35',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]': 'idade_36_45',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]': 'idade_46_ou_mais',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]': 'preta',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]': 'parda',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]': 'indigena',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]': 'amarela',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]': 'branca',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]': 'nao_respondeu_raca',
    'Em qual estado da federação você reside?': 'estado',
    'Há quanto tempo atua na área de tecnologia?': 'tempo_experiencia',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]': 'especialidade_dev_eng_software',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]': 'especialidade_devops',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]': 'especialidade_dados',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]': 'especialidade_suporte',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Infraestrutura]': 'especialidade_infraestrutura',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [QA]': 'especialidade_qa',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [SI]': 'especialidade_si',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]': 'especialidade_ux_ui',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]': 'especialidade_po',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]': 'especialidade_negocios',
    'Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)': 'criterios_vaga_afirmativa',
    'O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.': 'impacto_vaga_afirmativa',
    'Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.': 'sentimento_valorizacao',
    'Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.': 'oportunidades_desenvolvimento',
    'Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área. ': 'enfrentou_barreiras',
    'Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)': 'fatores_motivacao',
    'Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória.': 'opiniao_vagas_afirmativas'
}

# Remover espaços em branco no início/fim das chaves do dicionário column_mapping
column_mapping = {key.strip(): value for key, value in column_mapping.items()}

df = df_raw.rename(columns=column_mapping, errors='ignore')
print("✅ Colunas do DataFrame 'df' renomeadas.")

df.head(0)

✅ Colunas do DataFrame 'df' renomeadas.


,timestamp,idade_ate_25,idade_26_35,idade_36_45,idade_46_ou_mais,preta,parda,indigena,amarela,branca,nao_respondeu_raca,estado,tempo_experiencia,especialidade_dev_eng_software,especialidade_devops,especialidade_dados,especialidade_suporte,especialidade_infraestrutura,especialidade_qa,especialidade_si,especialidade_ux_ui,especialidade_po,especialidade_negocios,criterios_vaga_afirmativa,impacto_vaga_afirmativa,sentimento_valorizacao,oportunidades_desenvolvimento,enfrentou_barreiras,fatores_motivacao,opiniao_vagas_afirmativas,percepcao_racismo_limpo,idade_escolaridade,percepcao_lideranca_limpo,relevancia_mentoria_limpo,genero,raca


In [120]:
# 2.1 - Funções para consolidação das colunas de matriz
def consolidate_idade_escolaridade(row):
    for coluna in [c for c in row.index if c.startswith('idade_ate_25') or c.startswith('idade_26_35') or c.startwith('idade_36_45') or c.startswith('idade_46_ou_mais')]:
        value = row[col]
        if pd.notna(value) and str(value).strip() != '':
            idade = coluna
            escolaridade = value
            return pd.Series([idade, escolaridade])
            return value
    return np.nan

def consolidate_genero_raca_pair(row):
    for col in [c for c in row.index if c.startswith('preta') or c.startswith('parda') or c.startswith('indigena') or c.startswith('amarela') or c.startswith('branca') or c.startswith('não responder')]:
        value = row[col]
        if pd.notna(value) and str(value).strip() != '':
            raca = col
            genero = value
            return pd.Series([genero, raca])
    return pd.Series([np.nan, np.nan])



def consolidate_especialidade_senioridade(row):
    for col in [c for c in row.index if c.startswith('especialidade_dev_eng_software') or c.startswith('especialidade_devops') or c.startwith('especialidade_dados') or c.startswith('especialidade_suporte')
    or c.startswith('especialidade_infraestrutura') or c.startwith('especialidade_qa') or c.startswith('especialidade_si') or c.startswith('especialidade_ux_ui') or c.startwith('especialidade_po') or c.startswith('especialidade_negocios')]:
        value = row[col]
        if pd.notna(value) and str(value).strip() != '':
            especialidade = col
            senioridade = value
            return pd.Series([especialidade, senioridade])
            return value
    return np.nan

In [121]:
# 2.2 - Aplicação das funções para criar as novas colunas
df[['genero', 'raca']] = df.apply(consolidate_genero_raca_pair, axis=1)
df[['idade', 'escolaridade']] = df.apply(consolidate_idade_escolaridade, axis=1)
#df[['senioridade', 'especialidade']] = df.apply(consolidate_especialidade_senioridade, axis=1)

# Agora, fazemos a análise de frequência nas novas colunas
print("### Análise de Frequência - Perfil de Gênero e Raça ###")
print("-" * 40)
genero_counts = df['genero'].value_counts()
raca_counts = df['raca'].value_counts()
print("Distribuição de Gênero:")
print(genero_counts.to_string())
print("\n")
print("Distribuição de Raça:")
print(raca_counts.to_string())
print("\n")

# Agora, fazemos a análise de frequência nas novas colunas
print("### Análise de Frequência - Perfil de Idade e Escolaridade ###")
print("-" * 40)
idade_counts = df['idade'].value_counts()
escolaridade_counts = df['escolaridade'].value_counts()
print("Distribuição de idade:")
print(idade_counts.to_string())
print("\n")
print("Distribuição de Escolaridade:")
print(escolaridade_counts.to_string())
print("\n")

AttributeError: 'str' object has no attribute 'startwith'

In [ ]:

    # Re-run consolidation for idade_escolaridade and genero_raca after renaming
    # 2.2 Consolidar colunas de idade e escolaridade
    def consolidate_idade_escolaridade(row):
        # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
        idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
        idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
        idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
        idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
        prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


        if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
            return f"até 25 anos ({row[idade_ate_25_col_name]})"
        elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
            return f"26 a 35 ({row[idade_26_a_35_col_name]})"
        elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
            return f"36 a 45 ({row[idade_36_a_45_col_name]})"
        elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
            return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
        elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
            return row[prefiro_nao_informar_idade_col_name]
        else:
            return "Não informado"


    # Criar a coluna 'idade_escolaridade' using df_raw
    df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)
    print("✅ Coluna 'idade_escolaridade' consolidada.")


    # Consolidate columns of autodeclaração and escolaridade
    def consolidate_genero_raca(row):
        # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
        preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
        parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
        mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
        mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

        genero = ''
        # Also include other gender options if they exist and contribute to consolidation
        homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
        homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
        nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
        prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


        if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
            genero = row[mulher_cis_col_name]
        elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
             genero = row[mulher_trans_col_name]
        elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
             genero = row[homem_cis_col_name]
        elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
             genero = row[homem_trans_col_name]
        elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
             genero = row[nao_binarie_col_name]
        elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
             genero = row[prefiro_nao_informar_genero_col_name]


        if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
            return f"Preta ({genero})"
        elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
            return f"Parda ({genero})"
        elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
            return f"Indígena ({genero})"
        elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
            return f"Amarela ({genero})"
        elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
            return f"Branca ({genero})"
        elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
            return f"Não responder ({genero})"

        else:
            # If no race is marked, return "Outra" or just the gender if applicable
            return f"Outra ({genero})" if genero else "Outra"


    # Use the function to create the new consolidated column
    # Apply the function on the df_raw DataFrame and assign the result to a new column in df
    df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)
    print("✅ Coluna 'genero_raca' consolidada.")


    # Now re-run the analysis cells to confirm the columns are found and analyzed
    # 3.2 Idade e Escolaridade
    print("\n2. Distribuição de Idade e Nível de Escolaridade:")
    if 'idade_escolaridade' in df.columns:
        idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
        print(idade_escolaridade_counts.to_string())
    else:
        print("Column 'idade_escolaridade' not found in DataFrame.")
    print("\n")

    # 3.3 Tempo de Experiência
    print("3. Distribuição de Tempo de Experiência em Tecnologia:")
    if 'tempo_experiencia' in df.columns:
        tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
        print(tempo_experiencia_counts.to_string())
    else:
        print("Column 'tempo_experiencia' not found in DataFrame.")
    print("\n")

    # 3.4 Cargo Atual
    print("4. Distribuição de Cargo Atual:")
    if 'cargo_atual' in df.columns:
        cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
        print(cargo_atual_counts.to_string())
    else:
        print("Column 'cargo_atual' not found in DataFrame.")
    print("\n")

    # 3.5 Participação em Vagas Afirmativas
    print("5. Participação em Vagas Afirmativas:")
    if 'participou_vaga_afirmativa' in df.columns:
        participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
        print(participacao_vagas_counts.to_string())
    else:
         print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
    print("\n")

    # 6. Principais Desafios de Permanência (Respostas Múltiplas)
    print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
    if 'desafios_permanencia' in df.columns:
        desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
        desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
        print(desafios_counts.to_string())
    else:
        print("Column 'desafios_permanencia' not found in DataFrame.")
    print("\n")

else:
    print("Error: 'values' not found. Please run the cell to read the data from the Google Sheet first.")

SyntaxError: invalid syntax (ipython-input-1468356222.py, line 159)

In [ ]:
import pandas as pd
import numpy as np

# A partir do seu DataFrame 'df' já renomeado
# O código abaixo irá consolidar as colunas de matriz
def consolidate_all_matrix_questions(row):
    """
    Consolida as respostas de perguntas em formato de matriz em colunas únicas.
    """
    # 1. Consolidação de Gênero e Raça
    genero = np.nan
    raca = np.nan
    genero_raca_cols = [
        'preta', 'parda', 'indigena', 'amarela', 'branca', 'nao_respondeu_raca'
    ]
    for col in genero_raca_cols:
        if pd.notna(row[col]) and str(row[col]).strip() != '':
            raca = col
            genero = str(row[col]).strip()
            break

    # 2. Consolidação de Idade e Escolaridade
    idade_escolaridade = np.nan
    idade_cols = [
        'idade_ate_25', 'idade_26_35', 'idade_36_45', 'idade_46_ou_mais'
    ]
    for col in idade_cols:
        if pd.notna(row[col]) and str(row[col]).strip() != '':
            idade_escolaridade = str(row[col]).strip()
            break

    # 3. Consolidação de Especialidade e Senioridade
    senioridade = np.nan
    especialidade = np.nan
    senioridade_cols = [
        'senioridade_dev_eng_software', 'senioridade_devops', 'senioridade_dados',
        'senioridade_suporte', 'senioridade_infraestrutura', 'senioridade_qa',
        'senioridade_si', 'senioridade_ux_ui', 'senioridade_po', 'senioridade_negocios'
    ]
    for col in senioridade_cols:
        if pd.notna(row[col]) and str(row[col]).strip() != '':
            senioridade = str(row[col]).strip()
            # Extrai o nome da especialidade do nome da coluna
            especialidade = col.replace('senioridade_', '').replace('_', ' ').title()
            break

    return pd.Series([genero, raca, idade_escolaridade, senioridade, especialidade])

# Aplicação da função para criar as novas colunas de forma eficiente
df[['genero', 'raca', 'idade_escolaridade', 'senioridade', 'especialidade']] = df.apply(
    consolidate_all_matrix_questions, axis=1
)

# Exibe as novas colunas para ver o resultado
print("✅ Novas colunas de matrizes consolidadas com sucesso!")
print(df[['genero', 'raca', 'idade_escolaridade', 'senioridade', 'especialidade']].head())
print("\n")

# A partir daqui, você pode remover as colunas de matriz originais
# e continuar com o seu processo de criação de variáveis temáticas e ACM.

In [ ]:


# 2.3 - Criar as novas variáveis temáticas a partir das colunas de texto
df['opiniao_vagas_afirmativas'] = df['opiniao_vagas_afirmativas'].fillna('')
df['fatores_permanencia'] = df['fatores_permanencia'].fillna('')
df['desafios_permanencia'] = df['desafios_permanencia'].fillna('')
df['comentario_aberto'] = df['comentario_aberto'].fillna('')

df['tema_vagas_oportunidades'] = np.where(df['opiniao_vagas_afirmativas'].str.contains('vagas afirmativas|oportunidade|oportunidades|acesso', case=False, na=False), 'Sim', 'Não')
df['tema_cultura_ambiente'] = np.where(df['fatores_permanencia'].str.contains('cultura|diversidade|ambiente|inclusivo', case=False, na=False), 'Sim', 'Não')
df['tema_mentoria_suporte'] = np.where(df['fatores_permanencia'].str.contains('mentoria|suporte|apoio|desenvolvimento', case=False, na=False), 'Sim', 'Não')
df['tema_progressao_remuneracao'] = np.where(df['desafios_permanencia'].str.contains('salário|remuneração|promoção|crescimento|progressão', case=False, na=False), 'Sim', 'Não')

# 2.4 - Salvar o novo DataFrame com as variáveis adicionais e consolidadas ---
df.to_csv('dados_prontos_para_analise.csv', index=False)

print(f"✅ O arquivo 'dados_prontos_para_analise.csv' foi criado com sucesso, pronto para a Análise de Correspondência Múltipla.")

In [ ]:
# 2.3 - Criar as novas variáveis temáticas a partir das colunas de texto
df['comentario_aberto'] = df['comentario_aberto'].fillna('')
df['desafios_permanencia'] = df['desafios_permanencia'].fillna('')
df['opiniao_vagas_afirmativas'] = df['opiniao_vagas_afirmativas'].fillna('')
df['fatores_permanencia'] = df['fatores_permanencia'].fillna('')

df['tema_vagas_oportunidades'] = np.where(df['opiniao_vagas_afirmativas'].str.contains('vagas afirmativas|oportunidade|oportunidades|acesso', case=False, na=False), 'Sim', 'Não')
df['tema_cultura_ambiente'] = np.where(df['fatores_permanencia'].str.contains('cultura|diversidade|ambiente|inclusivo', case=False, na=False), 'Sim', 'Não')
df['tema_mentoria_suporte'] = np.where(df['fatores_permanencia'].str.contains('mentoria|suporte|apoio|desenvolvimento', case=False, na=False), 'Sim', 'Não')
df['tema_progressao_remuneracao'] = np.where(df['desafios_permanencia'].str.contains('salário|remuneração|promoção|crescimento|progressão', case=False, na=False), 'Sim', 'Não')

# --- 2.4 - Salvar o novo DataFrame com as variáveis adicionais e consolidadas ---
df.to_csv('dados_prontos_para_analise.csv', index=False)

print(f"✅ O arquivo 'dados_prontos_para_analise.csv' foi criado com sucesso, pronto para a Análise de Correspondência Múltipla.")

SyntaxError: invalid syntax (ipython-input-3287817275.py, line 6)

In [ ]:
# Tempo de Experiência
print("2. Distribuição do Tempo de Experiência em tecnologia:\n")
tempo_experiencia_counts = df_raw['Seu tempo de experiência em tecnologia é de:'].value_counts(normalize=True).mul(100).round(1)
print(tempo_experiencia_counts.to_string())
print("\n")

2. Distribuição do Tempo de Experiência em tecnologia:



KeyError: 'Seu tempo de experiência em tecnologia é de:'

In [ ]:
# Passo 2: Limpeza e Organização dos Dados
# Mapeamento para renomear colunas
# Esta etapa é crucial para a legibilidade do código

# 2.1 Renomear colunas para facilitar a manipulação
# Criando um dicionário de mapeamento de nomes longos para nomes curtos e significativos
column_mapping = {
    'Carimbo de data/hora': 'timestamp',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]': 'idade_ate_25',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]': 'idade_26_35',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]': 'idade_36_45',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]': 'idade_46_ou_mais',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]': 'preta',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]': 'parda',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]': 'indigena',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]': 'amarela',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]': 'branca',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]': 'mulher_cis_genero',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]': 'mulher_trans_genero',
    'Em qual estado do Brasil você reside?': 'estado',
    'Seu tempo de experiência em tecnologia é de:': 'tempo_experiencia',
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Qual o principal motivo de você ter participado de uma vaga afirmativa? (Se não participou, selecione a última opção)': 'motivo_vaga_afirmativa',
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho?': 'percepcao_racismo',
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho? [Concordo Totalmente]': 'percepcao_racismo_concordo_totalmente',
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho? [Concordo]': 'percepcao_racismo_concordo',
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho? [Neutro]': 'percepcao_racismo_neutro',
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho? [Discordo]': 'percepcao_racismo_discordo',
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho? [Discordo Totalmente]': 'percepcao_racismo_discordo_totalmente',
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia?': 'percepcao_lideranca',
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia? [Concordo Totalmente]': 'percepcao_lideranca_concordo_totalmente',
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia? [Concordo]': 'percepcao_lideranca_concordo',
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia? [Neutro]': 'percepcao_lideranca_neutro',
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia? [Discordo]': 'percepcao_lideranca_discordo',
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia? [Discordo Totalmente]': 'percepcao_lideranca_discordo_totalmente',
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional?': 'relevancia_mentoria',
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional? [Extremamente relevante]': 'relevancia_mentoria_extremamente_relevante',
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional? [Relevante]': 'relevancia_mentoria_relevante',
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional? [Neutro]': 'relevancia_mentoria_neutro',
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional? [Pouco relevante]': 'relevancia_mentoria_pouco_relevante',
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional? [Irrelevante]': 'relevancia_mentoria_irrelevante',
    'Onde você buscou ou busca por oportunidades de emprego na área de tecnologia?': 'fontes_oportunidades',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia',
    'Quais são os fatores mais importantes que a sua empresa/organização pode oferecer para garantir a permanência e o crescimento de mulheres negras na tecnologia?': 'fatores_permanencia',
    'Se desejar, deixe um comentário ou sugestão sobre o tema, ou alguma outra percepção que não foi mencionada no formulário.': 'comentario_aberto',
    # Adicione outros mapeamentos conforme necessário para as colunas que você quer manter/renomear
    # Mapeamentos para as colunas de especialidade/senioridade
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Análise de Dados]': 'analise_dados_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Ciência de Dados]': 'ciencia_dados_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Engenharia de Dados]': 'engenharia_dados_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Engenharia de Machine Learning]': 'engenharia_ml_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Desenvolvimento Back-end]': 'desenvolvimento_backend_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Desenvolvimento Front-end]': 'desenvolvimento_frontend_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Desenvolvimento Full-stack]': 'desenvolvimento_fullstack_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps/Cloud]': 'devops_cloud_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Segurança da Informação]': 'seguranca_informacao_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Gerenciamento de Projetos (Tech)]': 'gerenciamento_projetos_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]': 'ux_ui_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]': 'po_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]': 'negocios_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Outra]': 'outra_especialidade_senioridade',
    # Mapeamentos para as colunas de vagas afirmativas (critérios)
    'Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)': 'criterios_vaga_afirmativa',
    'O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.': 'impacto_vaga_afirmativa',
    # Mapeamentos para as colunas de percepção e motivação
    'Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.': 'sentimento_valorizacao',
    'Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.': 'oportunidades_desenvolvimento',
    'Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.': 'enfrentou_barreiras',
    'Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)': 'fatores_motivacao',
    'Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória.': 'opiniao_vagas_afirmativas'
}
df_raw = df_raw.rename(columns=column_mapping)
print("DataFrame criado e colunas principais renomeadas com sucesso!")
# Criar um novo DataFrame para os dados limpos
df_clean = pd.DataFrame()
df_raw.head()

DataFrame criado e colunas principais renomeadas com sucesso!


,timestamp,idade_ate_25_ensino_superior,idade_26_35_pos,idade_36_45_mestrado,idade_46_ou_mais_doutorado,raca_preta,raca_parda,indigena,amarela,branca,...,ux_ui_senioridade,po_senioridade,negocios_senioridade,"Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)",impacto_vaga_afirmativa,Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.,Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.,Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.,fatores_motivacao,opiniao_vagas_afirmativas
0,29/05/2025 14:25:27,,,,Pós-graduação,,Mulher(cis),,,,...,,,Estag/Trainee,Interseccional (direcionada a mulheres negras)...,Neutro,Neutro,Discordo,Discordo,Salário competitivo,None
1,30/05/2025 21:43:56,,Pós-graduação,,,Mulher(cis),,,,,...,,,,Não ingressei por vaga afirmativa,Não se aplica (não ingressei por vaga afirmati...,Concordo Totalmente,Concordo Totalmente,Neutro,"Reconhecimento profissional, Remuneração e pac...",Essenciais para melhorar a diversidade no trab...
2,31/05/2025 12:03:02,,Ensino Superior,,,Mulher(cis),,,,,...,,,,Interseccional (direcionada a mulheres negras),Concordo,Concordo,Concordo,Discordo,"Desafios técnicos e projetos estimulantes, Rem...",None
3,31/05/2025 17:40:18,,Ensino Superior,,,,Mulher(cis),,,,...,,,,"Raça (direcionada a pessoas negras), Gênero (d...",Concordo Totalmente,Concordo,Concordo Totalmente,Discordo,"Desafios técnicos e projetos estimulantes, Fle...",None
4,02/06/2025 10:09:19,Ensino Médio,,,,Mulher(cis),,,,,...,,,,Raça (direcionada a pessoas negras),Concordo,Concordo,Concordo Totalmente,Concordo,"Salário competitivo, Desafios técnicos e proje...",As vagas afirmativas são de extrema importânci...


In [ ]:
# Consolidar colunas de idade e escolaridade
df['idade_escolaridade'] = df_raw.filter(like='idade_').apply(
    lambda row: row.dropna().iloc[0] if not row.dropna().empty else None, axis=1
)

# Consolidar colunas de percepção e relevância
def consolidate_matrix_questions(df, prefix, new_col_name):
    """Consolida as colunas de uma pergunta matricial."""
    matrix_cols = [col for col in df.columns if col.startswith(prefix)]
    df[new_col_name] = df[matrix_cols].apply(
        lambda row: row.dropna().iloc[0].replace(prefix, '').replace('[', '').replace(']', '').strip() if not row.dropna().empty else None, axis=1
    )
    return df

df = consolidate_matrix_questions(df, 'percepcao_racismo', 'percepcao_racismo_limpo')
df = consolidate_matrix_questions(df, 'percepcao_lideranca', 'percepcao_lideranca_limpo')
df = consolidate_matrix_questions(df, 'relevancia_mentoria', 'relevancia_mentoria_limpo')

# Mover as colunas já limpas para o novo DataFrame
cols_to_move = [
    'timestamp', 'estado', 'tempo_experiencia', 'cargo_atual',
    'participou_vaga_afirmativa', 'motivo_vaga_afirmativa',
    'fontes_oportunidades', 'desafios_permanencia',
    'fatores_permanencia', 'comentario_aberto'
]
df = pd.concat([df, df_raw[cols_to_move]], axis=1)

# Salvar o novo DataFrame em um arquivo CSV limpo
#output_path = 'dados_limpos.csv'
#df_clean.to_csv(output_path, index=False)

print(f"✅ Análise concluída! O arquivo '{output_path}' foi criado com os dados limpos.")
print("\n--- Primeiras Análises na Nova Planilha Limpa ---\n")

# Agora podemos fazer análises mais precisas
print("1. Percepção sobre Racismo e Microagressões:")
percepcao_racismo_counts = df_clean['percepcao_racismo_limpo'].value_counts(normalize=True).mul(100).round(1)
print(percepcao_racismo_counts.to_string())
print("\n")

print("2. Percepção sobre Liderança:")
percepcao_lideranca_counts = df_clean['percepcao_lideranca_limpo'].value_counts(normalize=True).mul(100).round(1)
print(percepcao_lideranca_counts.to_string())
print("\n")

print("3. Relevância da Mentoria:")
relevancia_mentoria_counts = df_clean['relevancia_mentoria_limpo'].value_counts(normalize=True).mul(100).round(1)
print(relevancia_mentoria_counts.to_string())
print("\n")

# 4. Análise de respostas múltiplas (Desafios e Fatores de Permanência)
print("4. Principais Desafios de Permanência (multi-resposta):")
desafios_split = df_clean['desafios_permanencia'].str.split(',').explode().str.strip()
desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
print(desafios_counts.to_string())
print("\n")

print("5. Fatores mais importantes para Permanência (multi-resposta):")
fatores_split = df_clean['fatores_permanencia'].str.split(',').explode().str.strip()
fatores_counts = fatores_split.value_counts(normalize=True).mul(100).round(1)
print(fatores_counts.to_string())
print("\n")

KeyError: "None of [Index(['timestamp', 'estado', 'tempo_experiencia', 'cargo_atual',\n       'participou_vaga_afirmativa', 'motivo_vaga_afirmativa',\n       'fontes_oportunidades', 'desafios_permanencia', 'fatores_permanencia',\n       'comentario_aberto'],\n      dtype='object')] are in the [columns]"

In [ ]:
# 2.2 Consolidar colunas de autodeclaração e escolaridade
# Como o formulário gera uma coluna para cada opção, vamos criar uma única coluna para 'genero_raca'
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # Se nenhuma raça 'Preta' ou 'Parda' estiver marcada, retorne "Outro" ou apenas o gênero se aplicável
        return f"Outra ({genero})" if genero else "Outra"


# Usar a função para criar a nova coluna consolidada
# Aplicar a função no DataFrame df_raw e atribuir o resultado a uma nova coluna em df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

In [ ]:
# A mesma lógica se aplica à idade/escolaridade
def consolidate_idade_escolaridade(row):
    if pd.notna(row['idade_ate_25']):
        return row['idade_ate_25']
    elif pd.notna(row['idade_26_a_35']):
        return row['idade_26_a_35']
    elif pd.notna(row['idade_36_a_45']):
        return row['idade_36_a_45']
    elif pd.notna(row['idade_46_ou_mais']):
        return row['idade_46_ou_mais']
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade'
df['idade_escolaridade'] = df.apply(consolidate_idade_escolaridade, axis=1)

KeyError: 'idade_ate_25'

In [ ]:
# Passo 3: Análise Exploratória e Apresentação dos Resultados
print("### Análise Exploratória dos Dados ###\n")

# 3.1 Gênero e Raça
print("1. Distribuição de Raça/Cor e Gênero:")
genero_raca_counts = df['genero_raca'].value_counts(normalize=True).mul(100).round(1)
print(genero_raca_counts.to_string())
print("\n")

### Análise Exploratória dos Dados ###

1. Distribuição de Raça/Cor e Gênero:
genero_raca
Preta ()    51.2
Parda ()    29.3
Outra       19.5




In [ ]:
# 3.2 Idade e Escolaridade
print("2. Distribuição de Idade e Nível de Escolaridade:")
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

2. Distribuição de Idade e Nível de Escolaridade:


KeyError: 'idade_escolaridade'

In [ ]:
# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
print(tempo_experiencia_counts.to_string())
print("\n")

3. Distribuição de Tempo de Experiência em Tecnologia:


KeyError: 'tempo_experiencia'

In [ ]:
# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
print(cargo_atual_counts.to_string())
print("\n")

4. Distribuição de Cargo Atual:


KeyError: 'cargo_atual'

In [ ]:
# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
print(participacao_vagas_counts.to_string())
print("\n")

5. Participação em Vagas Afirmativas:


KeyError: 'participou_vaga_afirmativa'

In [ ]:
# Passo 4: Análise de Respostas Múltiplas (Exemplo)
# O questionário tem campos de resposta múltipla (ex: desafios_permanencia).
# A forma de análise é diferente. Vamos contar a frequência de cada opção.
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
desafios_split = df['desafios_permanencia'].str.split(',').explode().str.strip()
desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
print(desafios_counts.to_string())
print("\n")

6. Principais Desafios de Permanência (Respostas Múltiplas):


KeyError: 'desafios_permanencia'

In [ ]:
# Passo 2: Montar Google Drive (se necessário) e autenticar a API do Google Sheets

# Nome do arquivo JSON que você fez upload
# Certifique-se de que o nome do arquivo aqui corresponde ao nome do arquivo que você baixou e fez upload
SERVICE_ACCOUNT_FILE = 'nome_do_seu_arquivo_json.json' # <-- ATUALIZE AQUI COM O NOME DO SEU ARQUIVO JSON

# Montar o Google Drive. Este passo pode ser necessário se o arquivo JSON estiver no Drive.
# Se o arquivo JSON foi carregado diretamente no ambiente do Colab, este passo pode ser opcional,
# mas é mantido aqui por robustez caso o usuário opte por manter o arquivo no Drive.
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

# Carregar as credenciais da conta de serviço
try:
    if os.path.exists(SERVICE_ACCOUNT_FILE):
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets.readonly'])
        print("Credentials loaded successfully!")

        # Constrói o serviço Sheets
        service = build('sheets', 'v4', credentials=credentials)
        print("Google Sheets service built successfully!")
    else:
        print(f"ERROR: Credential file '{SERVICE_ACCOUNT_FILE}' not found.")
        print("Please ensure the JSON file is uploaded and the SERVICE_ACCOUNT_FILE variable is correct.")

except FileNotFoundError:
    print(f"ERROR: The credential file '{SERVICE_ACCOUNT_FILE}' was not found.")
    print("Please verify that you have uploaded the JSON file to Colab")
    print("and that the file name in the SERVICE_ACCOUNT_FILE variable is correct.")
except Exception as e:
    print(f"An error occurred while loading credentials or building the service: {e}")

# Verify that the service object is created
if 'service' in locals() and service:
    print("Authentication and service building completed.")
else:
    print("Authentication or service building failed.")

Mounted at /content/drive
Google Drive mounted successfully!
ERROR: Credential file 'nome_do_seu_arquivo_json.json' not found.
Please ensure the JSON file is uploaded and the SERVICE_ACCOUNT_FILE variable is correct.
Authentication and service building completed.


In [ ]:
# Passo 1: Importar bibliotecas necessárias
import os
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import auth
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.colab import userdata # Import userdata for accessing secrets

print("Todas as bibliotecas necessárias foram importadas com sucesso.")

In [ ]:
# Re-run the exploratory analysis code for 'cargo_atual', 'participou_vaga_afirmativa', and 'desafios_permanencia',
# including checks to see if the columns exist in df before attempting the analysis.

# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
# Adding a check just in case
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")


# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")


2. Distribuição de Idade e Nível de Escolaridade:
Column 'idade_escolaridade' not found in DataFrame.


3. Distribuição de Tempo de Experiência em Tecnologia:
Column 'tempo_experiencia' not found in DataFrame.


4. Distribuição de Cargo Atual:
Column 'cargo_atual' not found in DataFrame.


5. Participação em Vagas Afirmativas:
Column 'participou_vaga_afirmativa' not found in DataFrame.


6. Principais Desafios de Permanência (Respostas Múltiplas):
Column 'desafios_permanencia' not found in DataFrame.




In [ ]:
# Print the exact column names from df_raw to verify them once more.
print("Exact column names in df_raw:")
print(df_raw.columns.tolist())

# Print the current column_mapping dictionary to inspect the existing mappings.
print("\nCurrent column_mapping:")
print(column_mapping)

# Manually compare the printed df_raw column names with the keys in the column_mapping dictionary.
# Pay close attention to 'Qual seu cargo ou nível de atuação atual na empresa?',
# 'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?',
# 'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?',
# and '  Há quanto tempo atua na área de tecnologia?  ' (for tempo_experiencia).

# Update the column_mapping dictionary with any corrections identified.
# Based on previous attempts and df_raw output, re-confirm the exact keys.
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia',
    '  Há quanto tempo atua na área de tecnologia?  ': 'tempo_experiencia' # Keeping the exact name with spaces as seen in df_raw
})

# Re-run the renaming step using df = df_raw.rename(columns=column_mapping)
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns after re-running renaming with potentially corrected mapping:")
print(df.columns.tolist())


# Re-run the consolidation functions for idade_escolaridade and genero_raca.
# This is necessary because the df DataFrame was recreated in the previous step.

# Consolidate columns of idade and escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Consolidate columns of autodeclaração and escolaridade
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Re-execute the exploratory analysis code for 'cargo_atual', 'participou_vaga_afirmativa', and 'desafios_permanencia',
# including checks to see if the columns exist in df before attempting the analysis.

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")


# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Exact column names in df_raw:
['timestamp', 'idade_ate_25_ensino_superior', 'idade_26_35_pos', 'idade_36_45_mestrado', 'idade_46_ou_mais_doutorado', 'raca_preta', 'raca_parda', 'indigena', 'amarela', 'branca', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]', 'Em qual estado da federação você reside?', '  Há quanto tempo atua na área de tecnologia?  ', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua 

In [ ]:
# Re-run the analysis cells to confirm all columns are found and analyzed
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas:")
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):")
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")


2. Distribuição de Idade e Nível de Escolaridade:
idade_escolaridade
Não informado    100.0


3. Distribuição de Tempo de Experiência em Tecnologia:
tempo_experiencia
1 a 3 anos         43.9
4 a 6 anos         22.0
Menos de 1 ano     22.0
Mais de 10 anos     7.3
7 a 10 anos         4.9


4. Distribuição de Cargo Atual:
Column 'cargo_atual' not found in DataFrame.


5. Participação em Vagas Afirmativas:
Column 'participou_vaga_afirmativa' not found in DataFrame.


6. Principais Desafios de Permanência (Respostas Múltiplas):
Column 'desafios_permanencia' not found in DataFrame.




In [ ]:
# Print the exact column names from df_raw for a final verification
print("Final verification of columns in df_raw:")
print(df_raw.columns.tolist())

# Based on the output above, update the column_mapping with the precise names for the problematic columns
# Make absolutely sure the strings match exactly, including any leading/trailing spaces or special characters.
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia'
})


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after final, precise mapping correction.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas:")
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):")
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Final verification of columns in df_raw:
['timestamp', 'idade_ate_25_ensino_superior', 'idade_26_35_pos', 'idade_36_45_mestrado', 'idade_46_ou_mais_doutorado', 'raca_preta', 'raca_parda', 'indigena', 'amarela', 'branca', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]', 'Em qual estado da federação você reside?', '  Há quanto tempo atua na área de tecnologia?  ', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, 

In [ ]:
# Re-run the analysis cells to confirm the columns are found and analyzed
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas:")
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):")
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")


2. Distribuição de Idade e Nível de Escolaridade:
idade_escolaridade
26 a 35 (Ensino Superior)        26.8
26 a 35 (Pós-graduação)          17.1
até 25 anos (Ensino Superior)    14.6
46 ou mais (Pós-graduação)        7.3
até 25 anos (Ensino Médio)        7.3
36 a 45 (Pós-graduação)           7.3
até 25 anos (Pós-graduação)       7.3
26 a 35 (Mestrado)                7.3
26 a 35 (Ensino Médio)            2.4
36 a 45 (Ensino Superior)         2.4


3. Distribuição de Tempo de Experiência em Tecnologia:
tempo_experiencia
1 a 3 anos         43.9
4 a 6 anos         22.0
Menos de 1 ano     22.0
Mais de 10 anos     7.3
7 a 10 anos         4.9


4. Distribuição de Cargo Atual:
Column 'cargo_atual' not found in DataFrame.


5. Participação em Vagas Afirmativas:
Column 'participou_vaga_afirmativa' not found in DataFrame.


6. Principais Desafios de Permanência (Respostas Múltiplas):
Column 'desafios_permanencia' not found in DataFrame.




In [ ]:
# Print the exact column names from df_raw to verify again, focusing on problematic ones.
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Based on the df_raw output, identify and correct the mapping for the problematic columns.
# Let's re-check the exact original column names for 'cargo_atual', 'participou_vaga_afirmativa', and 'desafios_permanencia'

# It appears the mapping for 'tempo_experiencia' was corrected in the previous step.
# Let's focus on 'cargo_atual', 'participou_vaga_afirmativa', and 'desafios_permanencia'.

# From the df_raw output, the exact names are:
# 'Qual seu cargo ou nível de atuação atual na empresa?'
# 'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?'
# 'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?'

# Update the column_mapping with these exact names
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia'
})


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after re-running renaming with corrected mapping for remaining columns.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):")
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Update the column_mapping based on the exact names from df_raw
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia',
    '  Há quanto tempo atua na área de tecnologia?  ': 'tempo_experiencia'
})

# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after final mapping correction.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas):")
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")


DataFrame columns updated after final mapping correction.
Current columns in df:
['timestamp', 'idade_ate_25_original', 'idade_26_a_35_original', 'idade_36_a_45_original', 'idade_46_ou_mais_original', 'preta_raca_original', 'parda_raca_original', 'indigena_raca_original', 'amarela_raca_original', 'branca_raca_original', 'raca_nao_responder_original', 'estado', 'tempo_experiencia', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]', 'Qual sua especialidade e senioridade atual?\n(Por favor,

In [ ]:
# Print the exact column names from df_raw to verify again
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Check if the problematic columns exist in df after the last renaming
print("\nChecking for problematic columns in df after renaming:")
problematic_columns = ['tempo_experiencia', 'cargo_atual', 'participou_vaga_afirmativa', 'desafios_permanencia']
for col in problematic_columns:
    if col in df.columns:
        print(f"Column '{col}' found in df.")
    else:
        print(f"Column '{col}' NOT found in df.")

# Inspect the current column mapping for the problematic columns
print("\nCurrent column mapping for problematic columns:")
current_mapping = {
    'Em qual estado do Brasil você reside?': 'estado', # Corrected state column name based on df_raw
    'Seu tempo de experiência em tecnologia é de:': 'tempo_experiencia', # Corrected time experience column name based on df_raw
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual', # Corrected current role column name based on df_raw
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa', # Corrected affirmative vaga column name based on df_raw
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia' # Corrected challenges column name based on df_raw
}
for original_name, mapped_name in current_mapping.items():
     if original_name in df_raw.columns:
        print(f"'{original_name}' -> '{mapped_name}' (Original column exists in df_raw)")
     else:
         print(f"'{original_name}' -> '{mapped_name}' (Original column NOT found in df_raw) - Possible mapping issue")

# Based on the df_raw output and the checks, identify and correct the mapping if needed.
# It seems the column names from the previous step were slightly off.
# Let's re-update the column_mapping with the exact names observed in the df_raw output.
column_mapping.update({
    'Em qual estado da federação você reside?': 'estado',
    '  Há quanto tempo atua na área de tecnologia?  ': 'tempo_experiencia',
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia'
})


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after re-running renaming with corrected mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")


2. Distribuição de Idade e Nível de Escolaridade:
idade_escolaridade
26 a 35 (Ensino Superior)        26.8
26 a 35 (Pós-graduação)          17.1
até 25 anos (Ensino Superior)    14.6
46 ou mais (Pós-graduação)        7.3
até 25 anos (Ensino Médio)        7.3
36 a 45 (Pós-graduação)           7.3
até 25 anos (Pós-graduação)       7.3
26 a 35 (Mestrado)                7.3
26 a 35 (Ensino Médio)            2.4
36 a 45 (Ensino Superior)         2.4


3. Distribuição de Tempo de Experiência em Tecnologia:
Column 'tempo_experiencia' not found in DataFrame.


4. Distribuição de Cargo Atual:
Column 'cargo_atual' not found in DataFrame.


5. Participação em Vagas Afirmativas:
Column 'participou_vaga_afirmativa' not found in DataFrame.


6. Principais Desafios de Permanência (Respostas Múltiplas):
Column 'desafios_permanencia' not found in DataFrame.




In [ ]:
# Print the exact column names from df_raw to verify
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Re-initialize the column_mapping dictionary with all necessary mappings from scratch
column_mapping = {
    'Carimbo de data/hora': 'timestamp',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]': 'idade_ate_25_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]': 'idade_26_a_35_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]': 'idade_36_a_45_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]': 'idade_46_ou_mais_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]': 'idade_prefiro_nao_informar_original', # Added missing age column
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]': 'preta_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]': 'parda_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]': 'indigena_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]': 'amarela_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]': 'branca_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]': 'raca_nao_responder_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]': 'mulher_cis_genero_original', # Added missing gender columns for consolidation
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]': 'mulher_trans_genero_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]': 'homem_cis_genero_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]': 'homem_trans_genero_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]': 'nao_binarie_genero_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]': 'genero_prefiro_nao_informar_original',
    'Em qual estado do Brasil você reside?': 'estado', # Corrected state column name based on df_raw
    'Seu tempo de experiência em tecnologia é de:': 'tempo_experiencia', # Corrected time experience column name based on df_raw
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual', # Corrected current role column name based on df_raw
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa', # Corrected affirmative vaga column name based on df_raw
    'Qual o principal motivo de você ter participado de uma vaga afirmativa? (Se não participou, selecione a última opção)': 'motivo_vaga_afirmativa', # Added missing column
    'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho?': 'percepcao_racismo', # Added missing column
    'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia?': 'percepcao_lideranca', # Added missing column
    'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional?': 'relevancia_mentoria', # Added missing column
    'Onde você buscou ou busca por oportunidades de emprego na área de tecnologia?': 'fonte_oportunidades', # Added missing column
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia', # Corrected challenges column name based on df_raw
    'Quais são os fatores mais importantes que a sua empresa/organização pode oferecer para garantir a permanência e o crescimento de mulheres negras na tecnologia?': 'fatores_permanencia', # Added missing column
    'Se desejar, deixe um comentário ou sugestão sobre o tema, ou alguma outra percepção que não foi mencionada no formulário.': 'comentario_aberto', # Added missing column
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Análise de Dados]': 'analise_dados_senioridade', # Added specialty columns
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Ciência de Dados]': 'ciencia_dados_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Engenharia de Dados]': 'engenharia_dados_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Engenharia de Machine Learning]': 'engenharia_ml_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Desenvolvimento Back-end]': 'desenvolvimento_backend_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Desenvolvimento Front-end]': 'desenvolvimento_frontend_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Desenvolvimento Full-stack]': 'desenvolvimento_fullstack_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps/Cloud]': 'devops_cloud_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Segurança da Informação]': 'seguranca_informacao_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Gerenciamento de Projetos (Tech)]': 'gerenciamento_projetos_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]': 'ux_ui_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]': 'po_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]': 'negocios_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Outra]': 'outra_especialidade_senioridade',
    'Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam) ': 'criterios_vaga_afirmativa', # Corrected criteria column name
    'O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.': 'impacto_vaga_afirmativa', # Corrected impact column name
    'Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.': 'sentimento_valorizacao', # Corrected valorization column name
    'Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.': 'oportunidades_desenvolvimento', # Corrected opportunities column name
    'Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.': 'enfrentou_barreiras', # Corrected barriers column name
    'Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)': 'fatores_motivacao', # Corrected motivation column name
    'Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória.': 'opiniao_vagas_afirmativas' # Corrected opinion column name
}


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after re-initializing and re-running renaming.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells with checks
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Verify the exact column names from df_raw once more, focusing on the problematic ones.
print("Final check of columns in df_raw:")
print(df_raw.columns.tolist())

# Verify the current column mapping for the problematic columns.
print("\nCurrent column mapping for problematic columns:")
print({key: column_mapping.get(key) for key in ['Qual seu cargo ou nível de atuação atual na empresa?', 'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?', 'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?']})


# Re-run the renaming step with the current column_mapping to ensure df is correctly updated.
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns after re-running renaming:")
print(df.columns.tolist())

# Re-run the consolidation steps for idade_escolaridade and genero_raca.
# This is necessary because the df DataFrame was recreated in the previous step.

# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells with checks
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Final check of columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodecl

In [ ]:
# Print the exact column names from df_raw again to verify the remaining missing columns
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Based on the df_raw output, specifically check and update the mapping for the missing columns
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual', # Verify this name
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa', # Verify this name
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia' # Verify this name
})


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after re-running renaming with corrected mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"
    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Re-initialize the column_mapping dictionary with all necessary mappings from scratch
column_mapping = {
    'Carimbo de data/hora': 'timestamp',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]': 'idade_ate_25_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]': 'idade_26_a_35_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]': 'idade_36_a_45_original',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]': 'idade_46_ou_mais_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]': 'preta_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]': 'parda_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]': 'indigena_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]': 'amarela_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]': 'branca_raca_original',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]': 'raca_nao_responder_original',
    'Em qual estado da federação você reside?': 'estado',
    '  Há quanto tempo atua na área de tecnologia?  ': 'tempo_experiencia',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]': 'dev_engenheira_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]': 'devops_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]': 'dados_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]': 'suporte_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Infraestrutura]': 'infraestrutura_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [QA]': 'qa_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [SI]': 'si_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]': 'ux_ui_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]': 'po_senioridade',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]': 'negocios_senioridade',
    'Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam) ': 'criterios_vaga_afirmativa',
    'O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.': 'impacto_vaga_afirmativa',
    '  Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.  ': 'sentimento_valorizacao',
    '  Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.  ': 'oportunidades_desenvolvimento',
    'Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área. ': 'enfrentou_barreiras',
    'Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)': 'fatores_motivacao',
    'Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória.': 'opiniao_vagas_afirmativas'
}

# Re-run the renaming step with the re-initialized mapping
df = df_raw.rename(columns=column_mapping)

print("DataFrame columns updated after re-initializing and re-running renaming.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    # Also include other gender options if they exist and contribute to consolidation
    homem_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]'
    homem_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]'
    nao_binarie_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]'
    prefiro_nao_informar_genero_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]
    elif pd.notna(row.get(homem_cis_col_name)) and row.get(homem_cis_col_name) != '':
         genero = row[homem_cis_col_name]
    elif pd.notna(row.get(homem_trans_col_name)) and row.get(homem_trans_col_name) != '':
         genero = row[homem_trans_col_name]
    elif pd.notna(row.get(nao_binarie_col_name)) and row.get(nao_binarie_col_name) != '':
         genero = row[nao_binarie_col_name]
    elif pd.notna(row.get(prefiro_nao_informar_genero_col_name)) and row.get(prefiro_nao_informar_genero_col_name) != '':
         genero = row[prefiro_nao_informar_genero_col_name]


    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]') != '':
        return f"Indígena ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]') != '':
        return f"Amarela ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]') != '':
        return f"Branca ({genero})"
    elif pd.notna(row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]')) and row.get('Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]') != '':
        return f"Não responder ({genero})"

    else:
        # If no race is marked, return "Outra" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"


# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

DataFrame columns updated after re-initializing and re-running renaming.
Current columns in df:
['timestamp', 'idade_ate_25_original', 'idade_26_a_35_original', 'idade_36_a_45_original', 'idade_46_ou_mais_original', 'preta_raca_original', 'parda_raca_original', 'indigena_raca_original', 'amarela_raca_original', 'branca_raca_original', 'raca_nao_responder_original', 'estado', 'tempo_experiencia', 'dev_engenheira_senioridade', 'devops_senioridade', 'dados_senioridade', 'suporte_senioridade', 'infraestrutura_senioridade', 'qa_senioridade', 'si_senioridade', 'ux_ui_senioridade', 'po_senioridade', 'negocios_senioridade', 'criterios_vaga_afirmativa', 'impacto_vaga_afirmativa', 'sentimento_valorizacao', 'oportunidades_desenvolvimento', 'enfrentou_barreiras', 'fatores_motivacao', 'opiniao_vagas_afirmativas']

2. Distribuição de Idade e Nível de Escolaridade:
idade_escolaridade
26 a 35 (Ensino Superior)        26.8
26 a 35 (Pós-graduação)          17.1
até 25 anos (Ensino Superior)    14.6
46 o

In [ ]:
# Print the exact column names from df_raw again to verify the remaining missing columns
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Update the column_mapping based on the exact names from df_raw, focusing on the missing ones
# Based on the df_raw output, I will carefully update the mapping for the problematic columns.
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia'
})

# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after re-running renaming with corrected mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells with checks
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
if 'idade_escolaridade' in df.columns:
    idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
    print(idade_escolaridade_counts.to_string())
else:
    print("Column 'idade_escolaridade' not found in DataFrame.")
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
if 'desafios_permanencia' in df.columns:
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Inspect the unique values of potential columns for the missing data points
print("Unique values for potential 'participou_vaga_afirmativa' columns:")
print(df_raw['Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam) '].unique())
print(df_raw['O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.'].unique())
print(df_raw['Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?'].unique())

print("\nUnique values for potential 'cargo_atual' column:")
print(df_raw['Qual seu cargo ou nível de atuação atual na empresa?'].unique())

print("\nUnique values for potential 'desafios_permanencia' column:")
print(df_raw['Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?'].unique())

# Based on the unique values, I will update the column_mapping again if necessary.
# Then re-run the renaming and analysis.
# Assuming from the df_raw printout that the following mappings are correct:
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia',
    '  Há quanto tempo atua na área de tecnologia?  ': 'tempo_experiencia' # Keep this one as it worked before
})


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("\nDataFrame columns updated after re-running renaming with verified mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Unique values for potential 'participou_vaga_afirmativa' columns:
['Interseccional (direcionada a mulheres negras), Baixa renda'
 'Não ingressei por vaga afirmativa'
 'Interseccional (direcionada a mulheres negras)'
 'Raça (direcionada a pessoas negras), Gênero (direcionada a mulheres), Interseccional (direcionada a mulheres negras)'
 'Raça (direcionada a pessoas negras)'
 'Raça (direcionada a pessoas negras), Gênero (direcionada a mulheres)'
 'Não tenho clareza sobre o critério específico da vaga'
 'Gênero (direcionada a mulheres)'
 'Gênero (direcionada a mulheres), Baixa renda'
 'Não tenho clareza sobre o critério específico da vaga, Não ingressei por vaga afirmativa, Programa Desenvolve (sei que tinha % de vagas no programa para mulheres, mas na contratação não sei se foi um critério)'
 'Raça (direcionada a pessoas negras), Gênero (direcionada a mulheres), Pessoa com Deficiência (PCD), Baixa renda'
 'Raça (direcionada a pessoas negras), Gênero (direcionada a mulheres), Baixa renda'


KeyError: 'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?'

In [ ]:
# Update the column_mapping with the correct names for the persistently missing columns
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual', # Corrected name
    'Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam) ': 'criterios_vaga_afirmativa', # Found the correct name for criteria
    'O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.': 'impacto_vaga_afirmativa', # This one seems correct already, but double-checking
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa', # Corrected name
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia' # Corrected name
})


# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("DataFrame columns updated after final mapping correction.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

DataFrame columns updated after final mapping correction.
Current columns in df:
['timestamp', 'idade_ate_25_original', 'idade_26_a_35_original', 'idade_36_a_45_original', 'idade_46_ou_mais_original', 'preta_raca_original', 'parda_raca_original', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]', 'estado', '  Há quanto tempo atua na área de tecnologia?  ', 'Qual sua especialidade e senioridade atual?\n(Por favor, 

In [ ]:
# Print the exact column names from df_raw again to verify the remaining missing columns
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Update the column_mapping based on the exact names from df_raw, focusing on the missing ones
# Based on the df_raw output, it seems the keys in the original column_mapping were correct,
# but there might be leading/trailing spaces or hidden characters. Let's try stripping spaces.
column_mapping = {key.strip(): value for key, value in column_mapping.items()}

# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("DataFrame columns updated after re-running renaming with stripped mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Print the exact column names from df_raw again to verify the remaining missing columns
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Update the column_mapping based on the exact names from df_raw, focusing on the missing ones
column_mapping.update({
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual', # Re-verify this name
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa', # Re-verify this name
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia' # Re-verify this name
})

# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("DataFrame columns updated after re-running renaming with corrected mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")


2. Distribuição de Idade e Nível de Escolaridade:
idade_escolaridade
26 a 35 (Ensino Superior)        26.8
26 a 35 (Pós-graduação)          17.1
até 25 anos (Ensino Superior)    14.6
46 ou mais (Pós-graduação)        7.3
até 25 anos (Ensino Médio)        7.3
36 a 45 (Pós-graduação)           7.3
até 25 anos (Pós-graduação)       7.3
26 a 35 (Mestrado)                7.3
26 a 35 (Ensino Médio)            2.4
36 a 45 (Ensino Superior)         2.4


3. Distribuição de Tempo de Experiência em Tecnologia:
tempo_experiencia
1 a 3 anos         43.9
4 a 6 anos         22.0
Menos de 1 ano     22.0
Mais de 10 anos     7.3
7 a 10 anos         4.9


4. Distribuição de Cargo Atual:
Column 'cargo_atual' not found in DataFrame.


5. Participação em Vagas Afirmativas:
Column 'participou_vaga_afirmativa' not found in DataFrame.


6. Principais Desafios de Permanência (Respostas Múltiplas):
Column 'desafios_permanencia' not found in DataFrame.




In [ ]:
# Print the exact column names from df_raw to verify
print("Columns in df_raw:")
print(df_raw.columns.tolist())

# Update the column_mapping based on the exact names from df_raw
column_mapping.update({
    'Em qual estado da federação você reside?': 'estado',
    '  Há quanto tempo atua na área de tecnologia?  ': 'tempo_experiencia', # Corrected column name
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia'
})

# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("DataFrame columns updated after re-running renaming with corrected mapping.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade and genero_raca as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)

# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Columns in df_raw:
['Carimbo de data/hora', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]', 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação 

In [ ]:
# Add missing column mappings to the dictionary
column_mapping.update({
    'Em qual estado do Brasil você reside?': 'estado',
    'Seu tempo de experiência em tecnologia é de:': 'tempo_experiencia',
    'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
    'Você já participou de algum processo seletivo (trainee, estágio ou vagas afirmativas) destinado exclusivamente a mulheres negras na área de tecnologia?': 'participou_vaga_afirmativa',
    'Quais são os principais desafios de permanência e crescimento profissional que você enfrenta hoje na área de tecnologia?': 'desafios_permanencia'
})

# Re-run the renaming step with the updated mapping
df = df_raw.rename(columns=column_mapping)

print("DataFrame columns updated after re-running renaming.")
print("Current columns in df:")
print(df.columns.tolist())

# Re-run the consolidation for idade_escolaridade as the DataFrame was recreated
# 2.2 Consolidar colunas de idade e escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'

    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"

# Criar a coluna 'idade_escolaridade' using df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# Re-run the consolidation for genero_raca as the DataFrame was recreated
def consolidate_genero_raca(row):
    # Use os nomes originais das colunas de raça e gênero conforme aparecem no df_raw antes da renomeação
    preta_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]'
    parda_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]'
    mulher_cis_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(cis)]'
    mulher_trans_col_name = 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Mulher(trans)]'

    genero = ''
    if pd.notna(row.get(mulher_cis_col_name)) and row.get(mulher_cis_col_name) != '':
        genero = row[mulher_cis_col_name]
    elif pd.notna(row.get(mulher_trans_col_name)) and row.get(mulher_trans_col_name) != '':
         genero = row[mulher_trans_col_name]

    if pd.notna(row.get(preta_col_name)) and row.get(preta_col_name) != '':
        return f"Preta ({genero})"
    elif pd.notna(row.get(parda_col_name)) and row.get(parda_col_name) != '':
        return f"Parda ({genero})"
    else:
        # If no 'Preta' or 'Parda' race is marked, return "Outro" or just the gender if applicable
        return f"Outra ({genero})" if genero else "Outra"

# Use the function to create the new consolidated column
# Apply the function on the df_raw DataFrame and assign the result to a new column in df
df['genero_raca'] = df_raw.apply(consolidate_genero_raca, axis=1)


# Now re-run the analysis cells
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

DataFrame columns updated after re-running renaming.
Current columns in df:
['timestamp', 'idade_ate_25_original', 'idade_26_a_35_original', 'idade_36_a_45_original', 'idade_46_ou_mais_original', 'preta_raca_original', 'parda_raca_original', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]', 'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]', 'Em qual estado da federação você reside?', '  Há quanto tempo atua na área de tecnologia?  ', 'Qual sua especialidade e sen

In [ ]:
# Check the current columns in the dataframe
print("Current columns in df:")
print(df.columns.tolist())

# Update and execute the exploratory analysis cells with corrected column names
# 3.2 Idade e Escolaridade
print("\n2. Distribuição de Idade e Nível de Escolaridade:")
# This analysis was already done in the previous step and stored in df['idade_escolaridade']
idade_escolaridade_counts = df['idade_escolaridade'].value_counts(normalize=True).mul(100).round(1)
print(idade_escolaridade_counts.to_string())
print("\n")

# 3.3 Tempo de Experiência
print("3. Distribuição de Tempo de Experiência em Tecnologia:")
# Check if the column exists before attempting to access it
if 'tempo_experiencia' in df.columns:
    tempo_experiencia_counts = df['tempo_experiencia'].value_counts(normalize=True).mul(100).round(1)
    print(tempo_experiencia_counts.to_string())
else:
    print("Column 'tempo_experiencia' not found in DataFrame.")
print("\n")

# 3.4 Cargo Atual
print("4. Distribuição de Cargo Atual:")
# Check if the column exists before attempting to access it
if 'cargo_atual' in df.columns:
    cargo_atual_counts = df['cargo_atual'].value_counts(normalize=True).mul(100).round(1)
    print(cargo_atual_counts.to_string())
else:
    print("Column 'cargo_atual' not found in DataFrame.")
print("\n")

# 3.5 Participação em Vagas Afirmativas
print("5. Participação em Vagas Afirmativas:")
# Check if the column exists before attempting to access it
if 'participou_vaga_afirmativa' in df.columns:
    participacao_vagas_counts = df['participou_vaga_afirmativa'].value_counts(normalize=True).mul(100).round(1)
    print(participacao_vagas_counts.to_string())
else:
     print("Column 'participou_vaga_afirmativa' not found in DataFrame.")
print("\n")

# 6. Principais Desafios de Permanência (Respostas Múltiplas)
print("6. Principais Desafios de Permanência (Respostas Múltiplas):")
# Check if the column exists before attempting to access it
if 'desafios_permanencia' in df.columns:
    # Ensure no NaN values before splitting
    desafios_split = df['desafios_permanencia'].dropna().str.split(',').explode().str.strip()
    desafios_counts = desafios_split.value_counts(normalize=True).mul(100).round(1)
    print(desafios_counts.to_string())
else:
    print("Column 'desafios_permanencia' not found in DataFrame.")
print("\n")

Current columns in df:
['timestamp', 'Em qual estado da federação você reside?', '  Há quanto tempo atua na área de tecnologia?  ', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Infraestrutura]', 'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [QA]', 'Qual sua especialidade e senioridade 

In [ ]:
# 2.3 Excluir colunas redundantes após a consolidação
# Excluir as colunas originais que foram consolidadas or are no longer needed.
# Ensure to use the original column names that exist in the dataframe.
columns_to_drop = [
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]',
    'preta_raca_original',
    'parda_raca_original',
    'mulher_cis_genero_original',
    'mulher_trans_genero_original',
    'idade_ate_25_original',
    'idade_26_a_35_original',
    'idade_36_a_45_original',
    'idade_46_ou_mais_original'
    # Add any other original columns related to gender that might exist and were not handled in consolidation
    # based on inspecting the dataframe columns if needed.
    # For now, let's add other gender options that were not explicitly mapped but might be in the data.
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(cis)]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Homem(trans)]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não-binárie]',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'
]

# Drop columns if they exist in the dataframe
df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)


print("Colunas redundantes excluídas com sucesso!")
display(df.head())

Colunas redundantes excluídas com sucesso!


,timestamp,idade_ate_25_ensino_superior,idade_26_35_pos,idade_36_45_mestrado,idade_46_ou_mais_doutorado,raca_preta,raca_parda,indigena,amarela,branca,...,Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.,Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.,Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.,fatores_motivacao,opiniao_vagas_afirmativas,percepcao_racismo_limpo,percepcao_lideranca_limpo,relevancia_mentoria_limpo,idade_escolaridade,genero_raca
0,29/05/2025 14:25:27,,,,Pós-graduação,,Mulher(cis),,,,...,Neutro,Discordo,Discordo,Salário competitivo,None,NaN,NaN,NaN,Não informado,Outra
1,30/05/2025 21:43:56,,Pós-graduação,,,Mulher(cis),,,,,...,Concordo Totalmente,Concordo Totalmente,Neutro,"Reconhecimento profissional, Remuneração e pac...",Essenciais para melhorar a diversidade no trab...,NaN,NaN,NaN,Não informado,Outra
2,31/05/2025 12:03:02,,Ensino Superior,,,Mulher(cis),,,,,...,Concordo,Concordo,Discordo,"Desafios técnicos e projetos estimulantes, Rem...",None,NaN,NaN,NaN,Não informado,Outra
3,31/05/2025 17:40:18,,Ensino Superior,,,,Mulher(cis),,,,...,Concordo,Concordo Totalmente,Discordo,"Desafios técnicos e projetos estimulantes, Fle...",None,NaN,NaN,NaN,Não informado,Outra
4,02/06/2025 10:09:19,Ensino Médio,,,,Mulher(cis),,,,,...,Concordo,Concordo Totalmente,Concordo,"Salário competitivo, Desafios técnicos e proje...",As vagas afirmativas são de extrema importânci...,NaN,NaN,NaN,Não informado,Outra


In [ ]:
# 2.2 Consolidar colunas de idade e escolaridade
# A mesma lógica se aplica à idade/escolaridade
def consolidate_idade_escolaridade(row):
    # Use os nomes originais das colunas de idade/escolaridade conforme aparecem no df_raw
    idade_ate_25_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]'
    idade_26_a_35_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]'
    idade_36_a_45_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]'
    idade_46_ou_mais_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]'
    prefiro_nao_informar_idade_col_name = 'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [Prefiro não informar]'


    if pd.notna(row.get(idade_ate_25_col_name)) and row.get(idade_ate_25_col_name) != '':
        return f"até 25 anos ({row[idade_ate_25_col_name]})"
    elif pd.notna(row.get(idade_26_a_35_col_name)) and row.get(idade_26_a_35_col_name) != '':
        return f"26 a 35 ({row[idade_26_a_35_col_name]})"
    elif pd.notna(row.get(idade_36_a_45_col_name)) and row.get(idade_36_a_45_col_name) != '':
        return f"36 a 45 ({row[idade_36_a_45_col_name]})"
    elif pd.notna(row.get(idade_46_ou_mais_col_name)) and row.get(idade_46_ou_mais_col_name) != '':
        return f"46 ou mais ({row[idade_46_ou_mais_col_name]})"
    elif pd.notna(row.get(prefiro_nao_informar_idade_col_name)) and row.get(prefiro_nao_informar_idade_col_name) != '':
        return row[prefiro_nao_informar_idade_col_name]
    else:
        return "Não informado"


# Criar a coluna 'idade_escolaridade' usando df_raw
df['idade_escolaridade'] = df_raw.apply(consolidate_idade_escolaridade, axis=1)

# 2.3 Excluir colunas redundantes após a consolidação
# Excluir as colunas originais que foram consolidadas.
# Use os nomes originais das colunas conforme aparecem em df (que foram renomeados com _original)
df.drop(columns=[
    'preta_raca_original',
    'parda_raca_original',
    # Assuming other race/gender columns like Indígena, Amarela, Branca, Homem(cis), Homem(trans), Não-binárie, Prefiro não informar
    # were also included in the initial df_raw and were not consolidated into genero_raca
    # and should be dropped if they are empty or redundant.
    # Let's verify the current columns in df before dropping.
    # We should only drop columns that actually exist in the dataframe.
    'mulher_cis_genero_original',
    'mulher_trans_genero_original',
    'idade_ate_25_original',
    'idade_26_a_35_original',
    'idade_36_a_45_original',
    'idade_46_ou_mais_original'
    # Drop other original columns if they exist and are not needed for further analysis
    # based on the actual columns in df after the initial renaming.
], axis=1, inplace=True, errors='ignore') # Use errors='ignore' to avoid KeyError if a column doesn't exist


print("Colunas consolidadas e redundantes manipuladas com sucesso!")
display(df.head())


Colunas consolidadas e redundantes manipuladas com sucesso!


,timestamp,"Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]",Em qual estado da federação você reside?,Há quanto tempo atua na área de tecnologia?,"Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]",...,negocios_senioridade,"Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)",impacto_vaga_afirmativa,Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.,Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.,Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.,fatores_motivacao,opiniao_vagas_afirmativas,genero_raca,idade_escolaridade
0,29/05/2025 14:25:27,,,,,Sao,7 a 10 anos,,,,...,Estag/Trainee,Interseccional (direcionada a mulheres negras)...,Neutro,Neutro,Discordo,Discordo,Salário competitivo,None,Parda (),46 ou mais (Pós-graduação)
1,30/05/2025 21:43:56,,,,,Pernambuco,1 a 3 anos,Júnior,,,...,,Não ingressei por vaga afirmativa,Não se aplica (não ingressei por vaga afirmati...,Concordo Totalmente,Concordo Totalmente,Neutro,"Reconhecimento profissional, Remuneração e pac...",Essenciais para melhorar a diversidade no trab...,Preta (),26 a 35 (Pós-graduação)
2,31/05/2025 12:03:02,,,,,São Paulo,Menos de 1 ano,Júnior,,,...,,Interseccional (direcionada a mulheres negras),Concordo,Concordo,Concordo,Discordo,"Desafios técnicos e projetos estimulantes, Rem...",None,Preta (),26 a 35 (Ensino Superior)
3,31/05/2025 17:40:18,,,,,São Paulo,Menos de 1 ano,Júnior,,,...,,"Raça (direcionada a pessoas negras), Gênero (d...",Concordo Totalmente,Concordo,Concordo Totalmente,Discordo,"Desafios técnicos e projetos estimulantes, Fle...",None,Parda (),26 a 35 (Ensino Superior)
4,02/06/2025 10:09:19,,,,,São Paulo,1 a 3 anos,,,,...,,Raça (direcionada a pessoas negras),Concordo,Concordo,Concordo Totalmente,Concordo,"Salário competitivo, Desafios técnicos e proje...",As vagas afirmativas são de extrema importânci...,Preta (),até 25 anos (Ensino Médio)


In [ ]:
# Passo 3: Ler dados da planilha usando a API do Google Sheets

# Atualize com o ID da sua planilha e o nome da aba (ou range)
# O SPREADSHEET_ID está na URL da sua planilha, entre /d/ e /edit
SPREADSHEET_ID = '1sTiUrfRTddQLssaYDR38ouGwfmkSUP0rVY4ssO2dc2A'
RANGE_NAME = 'Respostas ao formulário 1!A1:AD42'

try:
    # Chamada à API para obter os dados
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('Nenhum dado encontrado no range especificado.')
    else:
        print(f'Dados lidos com sucesso! Número de linhas: {len(values)}')
        # Os dados lidos estão na variável 'values' (lista de listas)

except Exception as e:
    print(f"Ocorreu um erro ao ler os dados da planilha: {e}")


Dados lidos com sucesso! Número de linhas: 42


In [ ]:
# Passo 2: Montar Google Drive (se necessário) e autenticar a API do Google Sheets

# Nome do arquivo JSON que você fez upload
# Certifique-se de que o nome do arquivo aqui corresponde ao nome do arquivo que você baixou e fez upload
SERVICE_ACCOUNT_FILE = '/mba-usp-469620-7ff99acd80ca.json'

# Montar o Google Drive. Este passo pode ser necessário se o arquivo JSON estiver no Drive.
# Se o arquivo JSON foi carregado diretamente no ambiente do Colab, este passo pode ser opcional,
# mas é mantido aqui por robustez caso o usuário opte por manter o arquivo no Drive.
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

# Carregar as credenciais da conta de serviço
try:
    if os.path.exists(SERVICE_ACCOUNT_FILE):
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets.readonly'])
        print("Credentials loaded successfully!")

        # Constrói o serviço Sheets
        service = build('sheets', 'v4', credentials=credentials)
        print("Google Sheets service built successfully!")
    else:
        print(f"ERROR: Credential file '{SERVICE_ACCOUNT_FILE}' not found.")
        print("Please ensure the JSON file is uploaded and the SERVICE_ACCOUNT_FILE variable is correct.")

except FileNotFoundError:
    print(f"ERROR: The credential file '{SERVICE_ACCOUNT_FILE}' was not found.")
    print("Please verify that you have uploaded the JSON file to Colab")
    print("and that the file name in the SERVICE_ACCOUNT_FILE variable is correct.")
except Exception as e:
    print(f"An error occurred while loading credentials or building the service: {e}")

# Verify that the service object is created
if 'service' in locals() and service:
    print("Authentication and service building completed.")
else:
    print("Authentication or service building failed.")


Mounted at /content/drive
Google Drive mounted successfully!
Credentials loaded successfully!
Google Sheets service built successfully!
Authentication and service building completed.


In [ ]:
# Consolidar todas as importações necessárias no início do notebook

import os
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import auth
from google.oauth2 import service_account
from googleapiclient.discovery import build

print("Todas as bibliotecas necessárias foram importadas com sucesso.")

Todas as bibliotecas necessárias foram importadas com sucesso.


In [ ]:
# Instalar a biblioteca prince para Análise de Correspondência Múltipla (ACM)
!pip install prince

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import prince
import matplotlib.pyplot as plt

# Carregar o DataFrame limpo que criamos
#df = pd.read_csv('dados_limpos.csv')

# Definir as variáveis categóricas para a ACM
# Vamos selecionar as colunas que estão limpas e prontas
vars_para_acm = [
    'cargo_atual',
    'tempo_experiencia',
    'percepcao_racismo_limpo',
    'percepcao_lideranca_limpo',
    'relevancia_mentoria_limpo',
]

# Criar um novo DataFrame apenas com as variáveis de interesse
df_acm = df[vars_para_acm]

# Remover linhas com valores ausentes para garantir a qualidade da análise
df_acm = df_acm.dropna()

print("✅ DataFrame para Análise de Correspondência Múltipla preparado.")
print(f"Total de observações a serem analisadas: {len(df_acm)}\n")

# Passo 1: Instanciar e rodar o modelo de ACM
# n_components=2 para visualizar em 2D (eixos X e Y)
# n_iter=3 para maior precisão
acm = prince.MCA(
    n_components=2,
    n_iter=3,
    copy=True,
    random_state=42
)

# Fit o modelo com os dados
acm = acm.fit(df_acm)

# Passo 2: Analisar a inércia (autovalores e variância explicada)
print("### Resultados da Análise de Correspondência Múltipla ###")
print(f"Autovalores: {acm.eigenvalues_}")
print(f"Variância explicada por dimensão: {acm.explained_inertia_}")
print(f"Variância total explicada pelas 2 dimensões: {acm.explained_inertia_.sum():.2%}\n")
# A variância explicada nos mostra o quanto os dois eixos representam da variância total dos dados.
# Valores acima de 60-70% são considerados bons.

# Passo 3: Visualização do Mapa de Correspondência
# O método plot do 'prince' já gera o gráfico diretamente
print("✅ Gerando o Mapa de Correspondência...")
acm.plot_coordinates(
    X=df_acm,
    ax=None,
    figsize=(12, 10),
    show_row_points=False, # Não mostrar os pontos de cada respondente
    show_column_points=True, # Mostrar os pontos de cada categoria
    column_points_size=100,
    show_column_labels=True,
    legend_n_cols=1
)

plt.title('Mapa de Correspondência de Mulheres Negras em Tecnologia', fontsize=16, pad=20)
plt.show()

In [ ]:
import pandas as pd
import prince
import matplotlib.pyplot as plt

# Definir as variáveis categóricas para a ACM
# Vamos selecionar as colunas que estão limpas e prontas
# Certifique-se de que estas são as colunas que você deseja usar na ACM
vars_para_acm = [
    'cargo_atual',
    'tempo_experiencia',
    # Assuming these are created from original columns and are present after initial cleaning
    # If not, the logic to create them from original columns needs to be added here.
    'percepcao_racismo_limpo',
    'percepcao_lideranca_limpo',
    'relevancia_mentoria_limpo',
]

# --- Etapas de Preparação do DataFrame para a ACM (Combinadas aqui para garantir as colunas) ---

# Re-carregar os dados da planilha para garantir que estamos trabalhando com o df_raw original
# Certifique-se de que a célula de autenticação (Passo 2) foi executada antes desta.
try:
    # Atualize com o ID da sua planilha e o nome da aba (ou range)
    # O SPREADSHEET_ID está na URL da sua planilha, entre /d/ e /edit
    SPREADSHEET_ID = '1sTiUrfRTddQLssaYDR38ouGwfmkSUP0rVY4ssO2dc2A' # <-- ATUALIZE AQUI COM O ID DA SUA PLANILHA
    RANGE_NAME = 'Respostas ao formulário 1!A:AD' # <-- ATUALIZE AQUI COM O NOME DA ABA E O RANGE DOS DADOS (ex: Sheet1!A1:D10) - Ajustado para AD para pegar todas as colunas

    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('Nenhum dado encontrado no range especificado.')
    else:
        # A primeira linha da lista 'values' são os cabeçalhos
        headers = values[0]
        # As linhas restantes são os dados
        data = values[1:]
        # Criar o DataFrame df_raw a partir dos dados lidos
        df_raw_acm = pd.DataFrame(data, columns=headers)
        print("Dados da planilha recarregados com sucesso para a análise de ACM.")

except Exception as e:
    print(f"Ocorreu um erro ao recarregar os dados da planilha: {e}")
    df_raw_acm = None # Definir como None se houver erro na leitura


if df_raw_acm is not None:
    # Trabalhe em uma cópia para não alterar o df_raw_acm original se necessário (embora nesta célula não seja)
    df_temp = df_raw_acm.copy()

    # Limpar nomes das colunas em df_temp (remover espaços em branco no início/fim)
    df_temp.columns = df_temp.columns.str.strip()

    # 1. Definir e aplicar renomeação para as colunas relevantes para a ACM
    # Vamos incluir apenas os mapeamentos essenciais para as colunas usadas na ACM aqui
    column_mapping_acm = {
        'Qual seu cargo ou nível de atuação atual na empresa?': 'cargo_atual',
        'Há quanto tempo atua na área de tecnologia?': 'tempo_experiencia', # Nome corrigido sem espaços extras
        # Mapeamentos para as colunas que parecem ser a base das colunas '_limpo'
        # (Se você tiver os nomes exatos das colunas originais que geram as '_limpo', substitua abaixo)
        'Qual a sua percepção sobre a existência de microagressões e racismo no ambiente de trabalho?': 'percepcao_racismo_limpo', # Mapeando diretamente para o nome 'limpo'
        'Qual a sua percepção sobre a representatividade e visibilidade de mulheres negras em cargos de liderança na área de tecnologia?': 'percepcao_lideranca_limpo', # Mapeando diretamente para o nome 'limpo'
        'Qual o grau de relevância que você atribui a programas de mentoria e networking específicos para mulheres negras na sua trajetória profissional?': 'relevancia_mentoria_limpo', # Mapeando diretamente para o nome 'limpo'
    }

    # --- Debugging: Print columns before renaming ---
    print("\nColunas em df_temp antes da renomeação:")
    print(df_temp.columns.tolist())
    print("\nChaves no column_mapping_acm:")
    print(list(column_mapping_acm.keys()))
    # --- End Debugging ---

    # Aplicar renomeação para as colunas necessárias para ACM
    # Usamos errors='ignore' para não quebrar se alguma coluna original não for encontrada
    df_temp.rename(columns=column_mapping_acm, inplace=True, errors='ignore')

    # --- Debugging: Print columns after renaming ---
    print("\nColunas em df_temp após a renomeação:")
    print(df_temp.columns.tolist())
    # --- End Debugging ---

    # 2. Verificar e Garantir a Existência das Colunas para ACM
    # Após a renomeação, as colunas para ACM devem existir se os mapeamentos e nomes originais estiverem corretos.

    missing_acm_cols = [col for col in vars_para_acm if col not in df_temp.columns]
    if missing_acm_cols:
        print(f"\nERRO: As colunas necessárias para a análise de ACM ({missing_acm_cols}) não foram encontradas no DataFrame após a renomeação.")
        print("Por favor, verifique os nomes originais das colunas na sua planilha")
        print("e os mapeamentos no 'column_mapping_acm' dentro desta célula.")
        print("\nAnálise de ACM não pode prosseguir.")
    else:
        # 3. Criar um novo DataFrame apenas com as variáveis de interesse para a ACM
        df_acm = df_temp[vars_para_acm].copy()

        # 4. Remover linhas com valores ausentes para garantir a qualidade da análise
        df_acm = df_acm.dropna()

        print("✅ DataFrame para Análise de Correspondência Múltipla preparado.")
        print(f"Total de observações a serem analisadas: {len(df_acm)}\n")

        # --- Análise de Correspondência Múltipla (ACM) ---

        # Passo 1: Instanciar e rodar o modelo de ACM
        # n_components=2 para visualizar em 2D (eixos X e Y)
        # n_iter=3 para maior precisão
        acm = prince.MCA(
            n_components=2,
            n_iter=3,
            copy=True,
            random_state=42
        )

        # Fit o modelo com os dados
        acm = acm.fit(df_acm)

        # Passo 2: Analisar a inércia (autovalores e variância explicada)
        print("### Resultados da Análise de Correspondência Múltipla ###")
        print(f"Autovalores: {acm.eigenvalues_}")
        print(f"Variância explicada por dimensão: {acm.explained_inertia_}")
        print(f"Variância total explicada pelas 2 dimensões: {acm.explained_inertia_.sum():.2%}\n")
        # A variância explicada nos mostra o quanto os dois eixos representam da variância total dos dados.
        # Valores acima de 60-70% são considerados bons.

        # Passo 3: Visualização do Mapa de Correspondência
        # O método plot do 'prince' já gera o gráfico diretamente
        print("✅ Gerando o Mapa de Correspondência...")
        acm.plot_coordinates(
            X=df_acm,
            ax=None,
            figsize=(12, 10),
            show_row_points=False, # Não mostrar os pontos de cada respondente
            show_column_points=True, # Mostrar os pontos de cada categoria
            column_points_size=100,
            show_column_labels=True,
            legend_n_cols=1
        )

        plt.title('Mapa de Correspondência de Mulheres Negras em Tecnologia', fontsize=16, pad=20)
        plt.show()
else:
    print("Análise de ACM não pode prosseguir devido a erro na leitura dos dados da planilha.")